# Table of Contents
* [Description](#Description)
* [Python Import](#Python-Import)
* [Select Data Folder](#Select-Data-Folder)
* [Export Ascii File](#Export-Ascii-File)


# Description

This notebook creates an ASCII (text) file of the acquisition time of each file

The output file created will have the name of the data folder and will sit next to this data folder

Data folder:
```
    /data_set/0170113_lLL0_cycling_0120_000.tiff
    /data_set/0170113_lLL0_cycling_0120_001.tiff
    /data_set/0170113_lLL0_cycling_0120_002.tiff
    /data_set/0170113_lLL0_cycling_0120_003.tiff
    /data_set/0170113_lLL0_cycling_0120_004.tiff
    ...
```

**data_set.txt**
```
file_name, acquisition_time_computer_format, acquisition_time_user_format
0170113_LLZO_cycling_0120_0000.tiff,853193112.1120361,Mon Jan 13 17:05:12 1997
20170113_LLZO_cycling_0120_0001.tiff,853193237.0580895,Mon Jan 13 17:07:17 1997
20170113_LLZO_cycling_0120_0002.tiff,853193362.0047945,Mon Jan 13 17:09:22 1997
20170113_LLZO_cycling_0120_0003.tiff,853193486.9502357,Mon Jan 13 17:11:26 1997
20170113_LLZO_cycling_0120_0004.tiff,853193611.8544211,Mon Jan 13 17:13:31 1997
20170113_LLZO_cycling_0120_0005.tiff,853193736.7598045,Mon Jan 13 17:15:36 1997
20170113_LLZO_cycling_0120_0006.tiff,853193861.7127193,Mon Jan 13 17:17:41 1997
20170113_LLZO_cycling_0120_0007.tiff,853193986.6174998,Mon Jan 13 17:19:46 1997
20170113_LLZO_cycling_0120_0008.tiff,853194111.5720892,Mon Jan 13 17:21:51 1997
20170113_LLZO_cycling_0120_0009.tiff,853194236.4677708,Mon Jan 13 17:23:56 1997
20170113_LLZO_cycling_0120_0010.tiff,853194361.3547363,Mon Jan 13 17:26:01 1997
20170113_LLZO_cycling_0120_0011.tiff,853194486.3010912,Mon Jan 13 17:28:06 1997
...
```

# Python Import

In [6]:
from __code import custom_style
custom_style.style()

In [7]:
ipts_group = 'IPTS_17099'

import os
import glob
import time
import numpy as np

import pandas as pd

from ipywidgets.widgets import interact
from ipywidgets import widgets
from IPython.core.display import display, HTML

from PIL import Image
from PIL.ExifTags import TAGS

from __code import gui_widgets
from __code import config_handler
from __code import time_utility

display(HTML('<span style="font-size: 20px; color:blue">EXPLANATION: \
    we are importing all the python libraries</span>\
    <br><br><span style="font-size: 20px; color:blue"><b>Make sure you \
    wait until a cell is done executing before running another cell!</span>'))

In [8]:
%gui qt

# Select Data Folder

In [9]:
display(HTML('<span style="font-size: 20px; color:blue">EXPLANATION: select the location (folder) \
             of the data you want to load</span>'))

root_folder = config_handler.load_config(key='root_folder',
                                        default_value = '/HFIR/CG1D/IPTS_17099/',
                                        group=ipts_group)
data_folder = gui_widgets.gui_dname(dir=root_folder,
                                   message='Select the Data Folder ...')
if not data_folder == '':
    config_handler.save_config(key='data_folder', value=data_folder, group=ipts_group)
    config_handler.save_config(key='root_folder', value=os.path.dirname(data_folder), group=ipts_group)

    # get list of files
    list_of_files = np.sort(glob.glob(data_folder + '/*.tiff'))


In [10]:
display(HTML('<span style="font-size: 20px; color:blue">EXPLANATION: we are now loading the data.... BE PATIENT!</span>'))

w = widgets.IntProgress()
w.max = len(list_of_files)
display(w)

dict_acquisition_time = {}
file_to_remove = []
for _index, _file in enumerate(list_of_files):
    _short_file = os.path.basename(_file)
    try:
        dict_acquisition_time[_short_file] = time_utility.retrieve_time_stamp(_file)
    except KeyError:
        file_to_remove.append(_file)
    
    w.value = _index + 1
    
final_list_of_files = []
for _file in list_of_files:
    if _file in file_to_remove:
        continue
    final_list_of_files.append(_file)

list_of_files = final_list_of_files

# Export Ascii File

In [11]:
data_set = data_folder
display(HTML('<span style="font-size: 20px; color:blue">HELP: working with data set: ' + data_set + '</span>'))

In [12]:
display(HTML('<span style="font-size: 20px; color:blue">EXPLANATION: select where you want the ascii file \
    (that will containt the list of file names and their acquisition time) to be created.</span>'))
display(HTML('<span style="font-size: 20px; color:pink">WARNING: Make sure you have WRITE access to this folder!</span>'))

output_folder = config_handler.load_config(key='output_folder', group=ipts_group)
output_folder = gui_widgets.gui_dname(dir=output_folder,
                                   message='Select the Output Folder ...')
if output_folder:
    config_handler.save_config(key='output_folder', value=output_folder, group=ipts_group)   

    output_file_name = os.path.basename(data_folder) + '_time_spectra.txt'
    output_file_folder = output_folder
    full_output_file_name = os.path.join(output_file_folder, output_file_name)

    file_name_vs_time_stamp = pd.DataFrame(dict_acquisition_time).transpose()
    file_name_vs_time_stamp.to_csv(full_output_file_name, index_label=['file_name'])
    
    print("The following file has been created: {}".format(full_output_file_name))


The following file has been created: /Users/j35/Desktop/LLZO_DCcycle_0p4mA_to_1p2mA_time_spectra.txt
